In [ ]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from bokeh.plotting import output_notebook, figure, show
from bokeh.models import ColumnDataSource, Div, Select, Button, ColorBar, CustomJS
from bokeh.layouts import row, column, layout
from bokeh.transform import cumsum, linear_cmap
from bokeh.palettes import Blues8, Spectral3
from bokeh.plotting import figure, output_file, show

In [4]:
output_notebook()

Loading BokehJS ...

In [6]:
from plotly.offline import iplot
from plotly import tools
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
country = pd.read_csv('country.csv')

In [27]:
max(train['Date'])

'2020-03-24'

In [34]:
temp= train.groupby('Country/Region')[['ConfirmedCases','Fatalities']].sum().sort_values('ConfirmedCases',ascending=False)
temp['fatal_rate'] = temp['Fatalities'] / temp['ConfirmedCases']

In [35]:
temp.head()

,ConfirmedCases,Fatalities,fatal_rate
Country/Region,,,
China,3500461.0,121046.0,0.034580
Italy,571924.0,46856.0,0.081927
Iran,278992.0,16870.0,0.060468
Spain,232453.0,12464.0,0.053619
US,219412.0,2972.0,0.013545


In [57]:
temp2 = train.groupby(['Date','Country/Region'])['ConfirmedCases'].sum().reset_index()
temp2['size'] = temp2['ConfirmedCases'].pow(0.9)*10

In [58]:
covid1 = px.scatter_geo(temp2,locations='Country/Region',
              locationmode='country names',
              color='ConfirmedCases',
              size='size',
              hover_name='Country/Region',
              range_color=[1,200],
              projection='natural earth',
              animation_frame = 'Date',
              title='COVID19',
              color_continuous_scale="reds")

In [59]:
covid1.show()

# Time Series Prediction using LSTM

In [77]:
import tensorflow as tf

In [63]:
china = temp2[temp2['Country/Region']=='China']

In [65]:
china_cases = china['ConfirmedCases'].values

In [68]:
cc_mean = china_cases.mean()
cc_std = china_cases.std()

china_cases_scaled = (china_cases-cc_mean)/ cc_std

In [70]:
def univariate_data(dataset, start_index, end_index,history_size, target_size):
    data = []
    labels = []
    start_index = start_index + history_size
    
    if end_index is None:
        end_index = len(dataset) - target_size
        
    for i in range(start_index, end_index):
        indices = range(i-history_size, i)        
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [166]:
X_train,y_train = univariate_data(china_cases_scaled,0,int(len(china_cases_scaled)*0.8),10,0)
X_val,y_val = univariate_data(china_cases_scaled,int(len(china_cases_scaled)*0.8),None,10,0)

In [167]:
y_val

array([0.80087337, 0.80387148, 0.80676738])

In [168]:
batch_size = 32

train_data = tf.data.Dataset.from_tensor_slices((X_train,y_train))
train_data = train_data.batch(batch_size).repeat()

val_data = tf.data.Dataset.from_tensor_slices((X_val,y_val))
val_data = val_data.batch(batch_size).repeat()

In [169]:
lstm = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8,input_shape=X_train.shape[-2:]),
    tf.keras.layers.Dense(1)
])

lstm.compile(optimizer='adam',loss='mse')

In [170]:
epoch = 6
eval_interval = 100

In [171]:
lstm.fit(train_data,epochs=epoch,steps_per_epoch=eval_interval,
        validation_data=val_data,validation_steps=20,verbose=1)

Train for 100 steps, validate for 20 steps
Epoch 1/6
100/100 [==============================] - 5s 49ms/step - loss: 0.4414 - val_loss: 0.0110
Epoch 2/6
100/100 [==============================] - 1s 11ms/step - loss: 0.0563 - val_loss: 2.1426e-04
Epoch 3/6
100/100 [==============================] - 1s 14ms/step - loss: 0.0175 - val_loss: 9.2518e-04
Epoch 4/6
100/100 [==============================] - 1s 13ms/step - loss: 0.0066 - val_loss: 0.0017
Epoch 5/6
100/100 [==============================] - 1s 12ms/step - loss: 0.0054 - val_loss: 0.0017
Epoch 6/6
100/100 [==============================] - 1s 10ms/step - loss: 0.0052 - val_loss: 0.0016


In [172]:
scaled_pred = lstm.predict(X_val)
scaled_act = y_val

pred = (scaled_pred*cc_std)+cc_mean
act = (scaled_act*cc_std)+cc_mean

In [173]:
pred

array([[77987.23 ],
       [77981.766],
       [77975.58 ]], dtype=float32)

In [174]:
act

array([79070., 79158., 79243.])

In [175]:
china_act = np.append(china_cases,act)
china_pred = np.append(china_cases,pred.flatten())

In [176]:
china_act

array([  533.,   618.,   882.,  1321.,  1921.,  2649.,  5198.,  5692.,
        7637.,  9173., 11127., 15755., 18709., 22536., 26115., 29119.,
       32504., 35114., 38055., 40502., 42465., 42781., 57860., 64275.,
       66282., 68338., 70227., 71985., 72382., 72828., 73296., 74743.,
       74762., 74981., 75493., 75904., 76337., 76665., 77093., 77669.,
       77873., 77997., 78122., 78271., 78421., 78501., 78554., 78591.,
       78616., 78648., 78659., 78670., 78695., 78721., 78747., 78769.,
       78810., 78861., 78946., 78992., 79070., 79158., 79243., 79070.,
       79158., 79243.])

In [177]:
china_pred

array([  533.       ,   618.       ,   882.       ,  1321.       ,
        1921.       ,  2649.       ,  5198.       ,  5692.       ,
        7637.       ,  9173.       , 11127.       , 15755.       ,
       18709.       , 22536.       , 26115.       , 29119.       ,
       32504.       , 35114.       , 38055.       , 40502.       ,
       42465.       , 42781.       , 57860.       , 64275.       ,
       66282.       , 68338.       , 70227.       , 71985.       ,
       72382.       , 72828.       , 73296.       , 74743.       ,
       74762.       , 74981.       , 75493.       , 75904.       ,
       76337.       , 76665.       , 77093.       , 77669.       ,
       77873.       , 77997.       , 78122.       , 78271.       ,
       78421.       , 78501.       , 78554.       , 78591.       ,
       78616.       , 78648.       , 78659.       , 78670.       ,
       78695.       , 78721.       , 78747.       , 78769.       ,
       78810.       , 78861.       , 78946.       , 78992.    